# Extract profiles (Aschwanden, Truffer, and Fahnestock, 2016)

In [ ]:
import cartopy.crs as ccrs

import geopandas as gp
from joblib import Parallel, delayed
import numpy as np
import pandas as pd
from pathlib import Path
import pylab as plt
from matplotlib import colors, cm
from matplotlib.colors import LightSource
import matplotlib.ticker as mticker
from tqdm.auto import tqdm
import xarray as xr

from pypism import profiles
from pypism.profiles import process_profile
from pypism.utils import preprocess_nc
from pypism.hillshade import hillshade
from pypism.utils import qgis2cmap, tqdm_joblib, blend_multiply

## Resolution along profiles

In [ ]:
profile_resolution = 200 # m

## Load profiles and segmentize

In [ ]:
profiles_path = Path("../data/greenland-flux-gates.gpkg")
profiles_gp = gp.read_file(profiles_path).rename(columns={"id": "profile_id"})
geom = profiles_gp.segmentize(profile_resolution)
profiles_gp = gp.GeoDataFrame(profiles_gp, geometry=geom)

## Load observed velocities

In [ ]:
obs_file = Path("/Users/andy/Google Drive/My Drive/data/ITS_LIVE/GRE_G0240_0000.nc")
obs_ds = xr.open_dataset(obs_file)

## Load Ensemble Experiments

In [ ]:
from typing import Union
import numpy as np
import pylab as np
from shapely import get_coordinates
import cartopy.crs as ccrs
import cf_xarray.units  # pylint: disable=unused-import
import geopandas as gp
import numpy as np
import pint_xarray  # pylint: disable=unused-import
import pylab as plt
import seaborn as sns
import xarray as xr
from matplotlib import cm, colors
from matplotlib.colors import LightSource

from glacier_flow_tools.utils import blend_multiply, figure_extent, get_dataarray_extent, register_colormaps


## Extract all profiles

In [ ]:
cd ..

## Plot profiles

In [ ]:
run profiles/compute_profiles.py --thickness_url "~/Google Drive/My Drive/data/MCdataset/BedMachineGreenland-v5.nc" --n_jobs 4 --alpha 0.05 --result_dir profiles/2024_04_dev --velocity_url notebooks/GRE_G0240_0000.nc --profiles_url glacier_flow_tools/data/greenland-flux-gates.gpkg "~/Google Drive/My Drive/Projects/gris-calib/2022_09_*/state/gris_g1800m_v4_id_0_0_50.nc"

In [ ]:
def plot_glacier(
    profile_series: gp.GeoSeries,
    surface: xr.DataArray,
    overlay: xr.DataArray,
    result_dir: Union[str, Path],
    cmap="viridis",
    vmin: float = 10,
    vmax: float = 1500,
    ticks: Union[List[float], np.ndarray] = [10, 100, 250, 500, 750, 1500],
    font_size: float = 6,
    fig_width: float = 3.2
):
    """
    Plot a surface over a hillshade, add profile and correlation coefficient.

    This function plots a surface over a hillshade, adds a profile and correlation coefficient.
    The plot is saved as a PDF file in the specified result directory.

    Parameters
    ----------
    profile_series : gp.GeoSeries
        The profile to be plotted.
    surface : xr.DataArray
        The surface to be plotted over the hillshade.
    overlay : xr.DataArray
        The overlay to be added to the plot.
    result_dir : Union[str, Path]
        The directory where the result PDF file will be saved.
    cmap : str, optional
        The colormap to be used for the plot, by default "viridis".
    vmin : float, optional
        The minimum value for the colormap, by default 10.
    vmax : float, optional
        The maximum value for the colormap, by default 1500.
    ticks : Union[List[float], np.ndarray], optional
        The ticks to be used for the colorbar, by default [10, 100, 250, 500, 750, 1500].
    """
    plt.rcParams["font.size"] = font_size
    geom = getattr(profile_series, "geometry")
    geom_centroid = geom.centroid
    profile_centroid = gp.GeoDataFrame([profile_series], geometry=[geom_centroid])
    profile = gp.GeoDataFrame([profile_series], geometry=[geom])
    glacier_name = getattr(profile, "profile_name").values[0]
    exp_id = getattr(profile, "exp_id").values[0]
    geom = getattr(profile_centroid, "geometry")
    x, y = get_coordinates(geom).T
    x_c, y_c = np.round(x[0]), np.round(y[0])
    extent_slice = figure_extent(x_c, y_c)
    cartopy_crs = ccrs.NorthPolarStereo(central_longitude=-45, true_scale_latitude=70, globe=None)
    # Shade from the northwest, with the sun 45 degrees from horizontal
    light_source = LightSource(azdeg=315, altdeg=45)
    glacier_overlay = overlay.sel(extent_slice)
    glacier_surface = surface.interp_like(glacier_overlay)

    extent = get_dataarray_extent(glacier_overlay)
    norm = colors.Normalize(vmin=vmin, vmax=vmax)
    mapper = cm.ScalarMappable(norm=norm, cmap=cmap)

    v = mapper.to_rgba(glacier_overlay.to_numpy())
    z = glacier_surface.to_numpy()

    ar = 1.0  # initial aspect ratio for first trial
    wi = fig_width    # width in inches
    hi = wi * ar  # height in inches
    
    fig = plt.figure(figsize=(wi, hi))
    ax = fig.add_subplot(111, projection=cartopy_crs)
    rgb = light_source.shade_rgb(v, elevation=z, vert_exag=0.01, blend_mode=blend_multiply)
    # Use a proxy artist for the colorbar...
    im = ax.imshow(v, cmap=cmap, vmin=vmin, vmax=vmax)
    im.remove()
    corr = ax.imshow(
        v,
        vmin=0,
        vmax=1,
        cmap="RdYlGn",
    )
    corr.remove()
    ax.imshow(rgb, extent=extent, origin="upper", transform=cartopy_crs)
    profile.plot(ax=ax, color="k", lw=1)
    profile_centroid.plot(
        column="pearson_r",
        vmin=0,
        vmax=1,
        cmap="RdYlGn",
        markersize=50,
        legend=False,
        missing_kwds={},
        ax=ax,
        edgecolors="k",
        linewidths=0.2,
    )
    ax.set_title(glacier_name)
    ax.gridlines(
        draw_labels={"top": "x", "left": "y"},
        dms=True,
        xlocs=np.arange(-50, 0, 1),
        ylocs=np.arange(50, 88, 1),
        x_inline=False,
        y_inline=False,
        rotate_labels=20,
        ls="dotted",
        color="k",
    )

    ax.set_extent(extent, crs=cartopy_crs)
    fig.colorbar(im, ax=ax, shrink=0.5, pad=0.025, label=overlay.units, extend="both", ticks=ticks)
    fig.colorbar(
        corr, ax=ax, shrink=0.5, pad=0.025, label="Pearson $r$ (1)", orientation="horizontal", location="bottom"
    )
    plt.draw() 

    # Get proper ratio here
    xmin, xmax = ax.get_xbound()
    ymin, ymax = ax.get_ybound()
    y2x_ratio = (ymax-ymin)/(xmax-xmin)
    fig.set_figheight(wi * y2x_ratio)
    fig.tight_layout()
    fig.savefig(result_dir / Path(f"{glacier_name}_{exp_id}_speed.pdf"))
    plt.close()
    del fig


In [ ]:
p = stats_profiles.loc[0]

plot_glacier(p,
                surface_da,
                overlay_da,
                profile_result_dir,
                cmap=velocity_cmap,
            )

In [ ]:
plt.rcParams?

In [ ]:
os = obs_sims_profiles[0]

In [ ]:
p.plot.scatte

In [ ]:
from pathlib import Path
from glacier_flow_tools.utils import register_colormaps

In [ ]:
my_path = Path("/Users/andy/base/

In [ ]:
register_colormaps()


In [ ]:
import numpy as np
import pandas as pd
import geopandas as gp
from typing import Dict

In [ ]:
profiles_dict = {'profile_id': {0: 0, 1: 1},
 'profile_name': {0: 'Horizontal Gletscher', 1: 'Vertical'},
 'geometry': {0: LineString([[-10, 0], [10, 0]]),
  1: LineString([[0, -10],[0, 10]])}}
profiles_gp = gp.GeoDataFrame.from_dict(profiles_dict)

In [ ]:
from shapely import get_coordinates


In [ ]:
        nx = 201
        ny = 201
        x_min = -10
        x_max = 10
        y_min = -10
        y_max = 10
        x = np.linspace(x_min, x_max, nx)
        y = np.linspace(y_min, y_max, ny)
        X, Y = np.meshgrid(x, y)
    
        rng = np.random.default_rng(seed=42)
        # Directional vectors
        vx = np.zeros_like(X)
        vy = (y_max - Y**2) + rng.random(size=Y.shape)

In [ ]:
rng.random(size=Y.size).shape

In [ ]:
from shapely import get_coordinates

In [ ]:
 Path(str(files("glacier_flow_tools.data").joinpath("*.txt"))).parent.glob("*.txt")

In [ ]:

    nx = 201
    ny = 201
    x_min = -10
    x_max = 10
    y_min = -10
    y_max = 10
    x = np.linspace(x_min, x_max, nx)
    y = np.linspace(y_min, y_max, ny)
    X, Y = np.meshgrid(x, y)

    rng = np.random.default_rng(seed=42)
    # Directional vectors
    vx = (y_max**2 - Y**2) + rng.random(size=Y.shape)
    vy = np.zeros_like(X)

    coords = {
        "x": (
            ["x"],
            x,
            {
                "units": "m",
                "axis": "X",
                "standard_name": "projection_x_coordinate",
                "long_name": "x-coordinate in projected coordinate system",
            },
        ),
        "y": (
            ["y"],
            y,
            {
                "units": "m",
                "axis": "Y",
                "standard_name": "projection_y_coordinate",
                "long_name": "y-coordinate in projected coordinate system",
            },
        ),
    }

    ds = xr.Dataset(
        {
            "vx": xr.DataArray(
                data=vx,
                dims=["y", "x"],
                coords=coords,
                attrs={"standard_name": "velocity in x-direction", "units": "m/yr"},
            ),
            "vy": xr.DataArray(
                data=vy,
                dims=["y", "x"],
                coords=coords,
                attrs={"standard_name": "velocity in y-direction", "units": "m/yr"},
            ),
        },
        attrs={"Conventions": "CF-1.7"},
    )
    ds["Polar_Stereographic"] = int()
    ds.Polar_Stereographic.attrs["grid_mapping_name"] = "polar_stereographic"
    ds.Polar_Stereographic.attrs["false_easting"] = 0.0
    ds.Polar_Stereographic.attrs["false_northing"] = 0.0
    ds.Polar_Stereographic.attrs["latitude_of_projection_origin"] = 90.0
    ds.Polar_Stereographic.attrs["scale_factor_at_projection_origin"] = 1.0
    ds.Polar_Stereographic.attrs["standard_parallel"] = 70.0
    ds.Polar_Stereographic.attrs["straight_vertical_longitude_from_pole"] = -45
    ds.Polar_Stereographic.attrs["proj_params"] = "epsg:3413"
    quadratic_flow = ds

In [ ]:
files("glacier_flow_tools.data").joinpath("*.txt")

In [ ]:
    from glacier_flow_tools import profiles
    profiles_dict = {
        "profile_id": {0: 0, 1: 1},
        "profile_name": {0: "Horizontal Gletscher", 1: "Vertical"},
        "geometry": {0: LineString([[-10, 0], [10, 0]]), 1: LineString([[0, -10], [0, 10]])},
    }
    profiles_gp = gp.GeoDataFrame.from_dict(profiles_dict)
    geom = profiles_gp.segmentize(1.0)
    profiles_gp = gp.GeoDataFrame(profiles_gp, geometry=geom)
    profiles_gp = profiles_gp[["profile_id", "profile_name", "geometry"]]

    profile = profiles_gp.loc[[0]]
    geom = getattr(profile, "geometry")
    x_p, y_p = get_coordinates(geom).T
    profile_name = getattr(profile, "profile_name").values[0]
    profile_id = getattr(profile, "profile_id").values[0]

    kwargs: Dict = {}
    x_profile = quadratic_flow.profiles.extract_profile(x_p, y_p, profile_name=profile_name, profile_id=profile_id, **kwargs)

    profile = profiles_gp.loc[[1]]
    geom = getattr(profile, "geometry")
    x_p, y_p = get_coordinates(geom).T
    profile_name = getattr(profile, "profile_name").values[0]
    profile_id = getattr(profile, "profile_id").values[0]
    x_vy_true = np.array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0.])
    assert_array_almost_equal(x_profile.vy, x_vy_true)


In [ ]:
x_profile.vy

In [ ]:
kwargs.update({"compute_profile_normal": True})

In [ ]:
    y_profile = quadratic_flow.profiles.extract_profile(x_p, y_p, profile_name=profile_name, profile_id=profile_id, **kwargs)


In [ ]:
    profile = profiles_gp.loc[[1]]
    geom = getattr(profile, "geometry")
    x_p, y_p = get_coordinates(geom).T
    profile_name = getattr(profile, "profile_name").values[0]
    profile_id = getattr(profile, "profile_id").values[0]
    x_vy_true = np.array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0.])
    assert_array_almost_equal(x_profile.vy, x_vx_true)

    kwargs: Dict = {}
    y_profile = quadratic_flow.profiles.extract_profile(x_p, y_p, profile_name=profile_name, profile_id=profile_id, **kwargs)
array([  0.90858069,  19.51916723,  36.95104485,  51.28749629,
        64.28259608,  75.47610046,  84.02253044,  91.83760948,
        96.92277666,  99.93217176, 100.29915991,  99.22948628,
        96.05096098,  91.20227129,  84.15173917,  75.92502223,
        64.51036844,  51.41748735,  36.1520003 ,  19.53233673,
         0.23981163])

In [ ]:
    y_profile = quadratic_flow.profiles.extract_profile(x_p, y_p, profile_name=profile_name, profile_id=profile_id, **kwargs)


In [ ]:
normal_var = "v_normal"
normal_component_vars: dict = {"x": "vx", "y": "vy"}

In [ ]:
%load_ext rich

In [ ]:
y_profile.vx.to_numpy()

In [ ]:
    profile = profiles_gp.loc[[0]]
    geom = getattr(profile, "geometry")
    x_p, y_p = get_coordinates(geom).T
    profile_name = getattr(profile, "profile_name").values[0]
    profile_id = getattr(profile, "profile_id").values[0]

    kwargs: Dict = {}
    x_profile = quadratic_flow.profiles.extract_profile(x_p, y_p, profile_name=profile_name, profile_id=profile_id, **kwargs)


In [ ]:
    nx = 201
    ny = 201
    x_min = -10
    x_max = 10
    y_min = -10
    y_max = 10
    x = np.linspace(x_min, x_max, nx)
    y = np.linspace(y_min, y_max, ny)
    X, Y = np.meshgrid(x, y)

    rng = np.random.default_rng(seed=42)
    # Directional vectors
    vy = np.zeros_like(Y)
    vx = (y_max**2 - Y**2) + rng.random(size=Y.shape)


In [ ]:
def register_colormaps(path: Optional[Union[str, Path]] = None):
    """
    Register colormaps from text files in a specified directory or in the 'glacier_flow_tools.data' directory.

    This function reads all text files in the specified directory or in the 'glacier_flow_tools.data' directory if no directory is specified, converts each file to a colormap using the `qgis2cmap` function, and then registers the colormap using `plt.colormaps.register`.

    Each text file should define a colormap and its name should be the name of the colormap. The '.txt' extension is removed to get the name of the colormap.

    Parameters
    ----------
    path : str or Path, optional
        The directory where the colormap text files are located. If not provided, the 'glacier_flow_tools.data' directory is used.

    Examples
    --------
    >>> register_colormaps()
    >>> register_colormaps('/path/to/colormap/files')
    """
    if path is not None:
        cmap_files = Path(path).glob("*.txt")
    else:
        cmap_files = Path(str(files("glacier_flow_tools.data").joinpath("*.txt"))).parent.glob("*.txt")
    for cmap_file in cmap_files:
        name = cmap_file.name.removesuffix(".txt")
        cmap = qgis2cmap(cmap_file, name=name)
        plt.colormaps.register(cmap)


In [ ]:
from typing import Optional, Union

In [ ]:
plt.imshow(-(-Y**2))
plt.colorbar()

In [ ]:
import glacier_flow_tools.utils

In [ ]:
import numpy as np
from pathlib import Path
import pylab as plt
from importlib.resources import files
from glacier_flow_tools.utils import qgis2cmap, register_colormaps

In [ ]:
register_colormaps()
plt.colormaps.get_cmap("speed_colorblind")

In [ ]:
str(files("glacier_flow_tools.data"))

In [ ]:
    cmap_files = Path(str(files("glacier_flow_tools.data").joinpath("*.txt"))).parent.glob("*.txt")
    for cmap_file in cmap_files:
        print(cmap_file)
        name = cmap_file.name.removesuffix(".txt")
        print(name)
        cmap = qgis2cmap(cmap_file, name=name)
        print(cmap)
        plt.colormaps.register(cmap)


In [ ]:
cmap_file.name.removesuffix(".txt")

In [ ]:
plt.colormaps.register?

In [ ]:
Path(Path("GRE_G0240_0000.nc"))